![texto_descriptivo](portada.jpg)

### Importamos los módulos necesarios para el proyecto

In [71]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/w2_project_data_cleaning _and_wrangling/src/')

Mounted at /content/drive


In [72]:
import pandas as pd
import numpy as np 
import limpieza_texto as lt
import re
import seaborn as sns

In [73]:
from google.colab import files
# uploaded = files.upload()

In [74]:
lt.say_hello()

hello


### Cargamos el dataset

In [75]:
data = pd.read_csv("drive/MyDrive/Colab Notebooks/csv_data/attacks.csv",encoding = "ISO-8859-1")

In [76]:
data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


### Dataset study

In [77]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [78]:
data.shape

(25723, 24)

### Hypothesis

In [79]:
# 1. The white shark is the terrible bad guy.
# Variables: 'Species' and 'Fatal (Y/N)'

# 2. During the nighttime, sharks tend to be more fatal.
# Variables: 'Time' and 'Fatal (Y/N)'

# 3. Most shark attacks happen in US
# Variables: 'Country' 

### Limpieza de los datos

In [80]:
# Subset con las variable para el análisis de la hipotesis 1
data_1 = lt.get_subset(data, ['Species ', 'Fatal (Y/N)'])
data_1.head()

,Species,Fatal (Y/N)
0,White shark,N
1,NaN,N
2,NaN,N
3,2 m shark,N
4,"Tiger shark, 3m",N


In [81]:
# Eliminar los valores nulos 
data_1 = lt.remove_na(data_1, 'Species ')
data_1 = lt.remove_na(data_1, 'Fatal (Y/N)')
data_1.head()

,Species,Fatal (Y/N)
0,White shark,N
3,2 m shark,N
4,"Tiger shark, 3m",N
6,Tiger shark,Y
7,"Lemon shark, 3'",N


In [82]:
# Visualizar valores únicos para 'Species'
data_1['Species '].value_counts()

White shark                              161
Tiger shark                               73
Bull shark                                51
6' shark                                  40
4' shark                                  40
                                        ... 
1.8 m [6'] dogfish                         1
Wobbegong shark, 6'                        1
Species unknown, possible white shark      1
White shark, 5m to 6m                      1
Grey nurse shark, 8'                       1
Name: Species , Length: 1498, dtype: int64

In [83]:
# Función para agrupar valores en 'Species'
data_1['Shark species'] = lt.apply_filters(data_1)
data_1

,Species,Fatal (Y/N),Shark species
0,White shark,N,White
3,2 m shark,N,Other Species
4,"Tiger shark, 3m",N,Tiger
6,Tiger shark,Y,Tiger
7,"Lemon shark, 3'",N,Lemon
...,...,...,...
6276,12' tiger shark,Y,Tiger
6293,Blue pointer,Y,Blue
6294,Blue pointer,Y,Blue
6295,Blue pointers,Y,Blue


In [84]:
data_1['Shark species'].unique()

array(['White', 'Other Species', 'Tiger', 'Lemon', 'Bull', 'Gray reef',
       'Nurse', 'Blacktip reef', 'Galapagos', 'Blue', 'Spinner',
       'Porbeagle', 'Sandtiger', 'Caribbean reef', 'Broadnose sevengill',
       'Dog', 'Mako', 'Whale', 'Hammerhead', 'Raggedtooth', 'Goblin',
       'Sevengill', 'Zambezi', 'dusky', 'Gray', 'Brown', 'Carpet',
       'Shovelnose'], dtype=object)

In [85]:
# Eliminar columna 'Species'
data_1 = lt.drop_column(data_1, 'Species ')
data_1.head(5)

,Fatal (Y/N),Shark species
0,N,White
3,N,Other Species
4,N,Tiger
6,Y,Tiger
7,N,Lemon


In [86]:
# Visualización de valores únicos para 'Fatal (Y/N)'
data_1['Fatal (Y/N)'].value_counts()

N          2497
Y           427
UNKNOWN      17
 N            5
M             1
2017          1
Name: Fatal (Y/N), dtype: int64

In [87]:
data_1['Fatal (Y/N)'].unique()

array(['N', 'Y', 'M', '2017', 'UNKNOWN', ' N'], dtype=object)

In [88]:
# Función para identificar los missing_values de la columna 'Fatal (Y/N)'
# Crear nueva columna para filtrar como True los missing_values
data_1['delete_missing_values_fatal'] = lt.apply_missing_fatal(data_1)
print(data_1)

     Fatal (Y/N)  Shark species  delete_missing_values_fatal
0              N          White                        False
3              N  Other Species                        False
4              N          Tiger                        False
6              Y          Tiger                        False
7              N          Lemon                        False
...          ...            ...                          ...
6276           Y          Tiger                        False
6293           Y           Blue                        False
6294           Y           Blue                        False
6295           Y           Blue                        False
6296           Y      Sandtiger                        False

[2948 rows x 3 columns]


In [89]:
data_1['Fatal (Y/N)'].unique()

(2948, 3)


In [90]:

# Eliminar las filas cuyo valor sea True en la columna 'delete_missing_values_fatal'
data_1 = lt.drop_true_rows_by_column(data_1, 'delete_missing_values_fatal')
print(data_1.head())


  Fatal (Y/N)  Shark species  delete_missing_values_fatal
0           N          White                        False
3           N  Other Species                        False
4           N          Tiger                        False
6           Y          Tiger                        False
7           N          Lemon                        False


In [91]:
print(data_1['Fatal (Y/N)'].unique())

['N' 'Y']
(2924, 3)


In [92]:
data_1['Fatal (Y/N)'].value_counts()

N    2497
Y     427
Name: Fatal (Y/N), dtype: int64

In [93]:
# Eliminar la columna 'delete_missing_values_fatal'
data_1 = lt.drop_column(data_1, 'delete_missing_values_fatal')
data_1.head()

,Fatal (Y/N),Shark species
0,N,White
3,N,Other Species
4,N,Tiger
6,Y,Tiger
7,N,Lemon


In [94]:
data_1.shape

(2924, 2)

In [95]:
# Exportar dataset 1
data_1.to_csv("drive/MyDrive/Colab Notebooks/csv_data/attacks_hypothesis1.csv", index = False, encoding="ISO-8859-1")

In [96]:
# Subset con las variable para el análisis de la hipotesis 2
data_2 = lt.get_subset(data, ['Fatal (Y/N)', 'Time'])
data_2.head()

,Fatal (Y/N),Time
0,N,18h00
1,N,14h00 -15h00
2,N,07h45
3,N,NaN
4,N,NaN


In [97]:
# Eliminar los valores nulos 
data_2 = lt.remove_na(data_2, 'Time')
data_2 = lt.remove_na(data_2, 'Fatal (Y/N)')
data_2.head()

,Fatal (Y/N),Time
0,N,18h00
1,N,14h00 -15h00
2,N,07h45
6,Y,Late afternoon
8,N,17h00


In [98]:
# Visualización de valores únicos para 'Time'
data_2['Time'].value_counts()

Afternoon        176
11h00            117
Morning          117
12h00            107
15h00            104
                ... 
13h37              1
>17h30             1
Early Morning      1
14h00  -15h00      1
19h00 / 20h00      1
Name: Time, Length: 351, dtype: int64

In [99]:
data_2['Time'].unique()

array(['18h00', '14h00  -15h00', '07h45', 'Late afternoon', '17h00',
       '14h00', 'Morning', '15h00', '08h15', '11h00', '10h30', '10h40',
       '16h50', '07h00', '09h30', 'Afternoon', '21h50', '09h40', '08h00',
       '17h35', '15h30', '07h30', '19h00, Dusk', 'Night', '16h00',
       '15h01', '12h00', '13h45', '23h30', '09h00', '14h30', '18h30',
       '12h30', '16h30', '18h45', '06h00', '10h00', '10h44', '13h19',
       '13h30', '10h45', '11h20', '11h45', '08h30', '15h45',
       'Shortly before 12h00', '17h34', '17h10', '11h15', '17h45',
       '13h00', '10h20', '13h20', '02h00', '09h50', '11h30', '17h30',
       '9h00', '10h43', 'After noon', '15h15', '15h40', '19h05', '1300',
       '14h30 / 15h30', '22h00', '16h20', '14h34', '15h25', '14h55',
       '17h46', 'Morning ', '15h49', '19h00', 'Midnight', '09h30 / 10h00',
       '10h15', '18h15', '04h00', '14h50', '13h50', '10h25',
       '10h45-11h15', '16h45', '15h52', '06h15', '14h40', '08h45',
       'Evening', '12h10', '18h05',

In [100]:
# Función para agrupar valores en 'Time'
# Crear nueva columna para indicar lel resultado de new_values_time
data_2['Day period'] = lt.apply_time_correction(data_2)
print(data_2)

     Fatal (Y/N)            Time Day period
0              N           18h00  Afternoon
1              N   14h00  -15h00  Afternoon
2              N           07h45    Morning
6              Y  Late afternoon  Afternoon
8              N           17h00  Afternoon
...          ...             ...        ...
6241           N       Afternoon  Afternoon
6242           N       Afternoon  Afternoon
6243           N       Afternoon  Afternoon
6259           Y  Late afternoon  Afternoon
6265           Y       Afternoon  Afternoon

[2802 rows x 3 columns]


In [101]:
data_2['Day period'].value_counts()

Afternoon    1690
Morning       937
Night         137
--             38
Name: Day period, dtype: int64

In [102]:
# Eliminar las filas cuyo valor sea '--' en la columna 'Day period'
data_2 = lt.drop_dashed_rows_by_column(data_2, 'Day period')
data_2.head()

,Fatal (Y/N),Time,Day period
0,N,18h00,Afternoon
1,N,14h00 -15h00,Afternoon
2,N,07h45,Morning
6,Y,Late afternoon,Afternoon
8,N,17h00,Afternoon


In [103]:
# Eliminar la columna 'Time'
data_2 = lt.drop_column(data_2, 'Time')
data_2.head()

,Fatal (Y/N),Day period
0,N,Afternoon
1,N,Afternoon
2,N,Morning
6,Y,Afternoon
8,N,Afternoon


In [104]:
# Visualización de valores únicos para 'Fatal (Y/N)'
data_2['Fatal (Y/N)'].value_counts()

N          2269
Y           483
UNKNOWN       7
 N            3
M             1
2017          1
Name: Fatal (Y/N), dtype: int64

In [105]:
data_2['Fatal (Y/N)'].unique()

array(['N', 'Y', 'M', '2017', 'UNKNOWN', ' N'], dtype=object)

In [106]:
# Función para identificar los missing_values de la columna 'Fatal (Y/N)'
data_2['delete_missing_values_fatal'] = lt.apply_missing_fatal(data_2)
print(data_2)

     Fatal (Y/N) Day period  delete_missing_values_fatal
0              N  Afternoon                        False
1              N  Afternoon                        False
2              N    Morning                        False
6              Y  Afternoon                        False
8              N  Afternoon                        False
...          ...        ...                          ...
6241           N  Afternoon                        False
6242           N  Afternoon                        False
6243           N  Afternoon                        False
6259           Y  Afternoon                        False
6265           Y  Afternoon                        False

[2764 rows x 3 columns]


In [107]:
# Eliminar las filas cuyo valor sea True en la columna 'delete_missing_values_fatal'
data_2 = lt.drop_true_rows_by_column(data_2, 'delete_missing_values_fatal')
data_2.head()

,Fatal (Y/N),Day period,delete_missing_values_fatal
0,N,Afternoon,False
1,N,Afternoon,False
2,N,Morning,False
6,Y,Afternoon,False
8,N,Afternoon,False


In [108]:
# Eliminar la columna 'delete_missing_values_fatal'
data_2 = lt.drop_column(data_2, 'delete_missing_values_fatal')
data_2.head()

,Fatal (Y/N),Day period
0,N,Afternoon
1,N,Afternoon
2,N,Morning
6,Y,Afternoon
8,N,Afternoon


In [109]:
# Exportar dataset 2
data_2.to_csv("drive/MyDrive/Colab Notebooks/csv_data/attacks_hypothesis2.csv", index = False, encoding="ISO-8859-1")

In [110]:
# Subset con las variable para el análisis de la hipotesis 3
data_3 = lt.get_subset(data, ['Country'])
data_3.head()

,Country
0,USA
1,USA
2,USA
3,AUSTRALIA
4,MEXICO


In [111]:
# Eliminar los valores nulos 
data_3 = lt.remove_na(data_3, 'Country')
print(data_3)

                 Country
0                    USA
1                    USA
2                    USA
3              AUSTRALIA
4                 MEXICO
...                  ...
6297           AUSTRALIA
6298           AUSTRALIA
6299                 USA
6300              PANAMA
6301  CEYLON (SRI LANKA)

[6252 rows x 1 columns]


In [112]:
# Visualizar valores únicos para 'Country'
data_3['Country'].value_counts()

USA                 2229
AUSTRALIA           1338
SOUTH AFRICA         579
PAPUA NEW GUINEA     134
NEW ZEALAND          128
                    ... 
PARAGUAY               1
ARUBA                  1
ALGERIA                1
INDIAN OCEAN?          1
ANDAMAN ISLANDS        1
Name: Country, Length: 212, dtype: int64

In [113]:
data_3['Country'].unique()

array(['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'ENGLAND', 'SOUTH AFRICA',
       'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS', 'NEW CALEDONIA',
       'ECUADOR', 'MALAYSIA', 'LIBYA', 'CUBA', 'MAURITIUS', 'NEW ZEALAND',
       'SPAIN', 'SAMOA', 'SOLOMON ISLANDS', 'JAPAN', 'EGYPT',
       'ST HELENA, British overseas territory', 'COMOROS', 'REUNION',
       'FRENCH POLYNESIA', 'UNITED KINGDOM', 'UNITED ARAB EMIRATES',
       'PHILIPPINES', 'INDONESIA', 'CHINA', 'COLUMBIA', 'CAPE VERDE',
       'Fiji', 'DOMINICAN REPUBLIC', 'CAYMAN ISLANDS', 'ARUBA',
       'MOZAMBIQUE', 'FIJI', 'PUERTO RICO', 'ITALY', 'ATLANTIC OCEAN',
       'GREECE', 'ST. MARTIN', 'FRANCE', 'PAPUA NEW GUINEA',
       'TRINIDAD & TOBAGO', 'KIRIBATI', 'ISRAEL', 'DIEGO GARCIA',
       'TAIWAN', 'JAMAICA', 'PALESTINIAN TERRITORIES', 'GUAM',
       'SEYCHELLES', 'BELIZE', 'NIGERIA', 'TONGA', 'SCOTLAND', 'CANADA',
       'CROATIA', 'SAUDI ARABIA', 'CHILE', 'ANTIGUA', 'KENYA', 'RUSSIA',
       'TURKS & CAICOS', 'UNITED ARA

In [114]:
# Función para sustituir string en las celdas
data_3 = lt.replace_countries(data_3)
print(data_3)

                 Country
0                    USA
1                    USA
2                    USA
3              AUSTRALIA
4                 MEXICO
...                  ...
6297           AUSTRALIA
6298           AUSTRALIA
6299                 USA
6300              PANAMA
6301  CEYLON (SRI LANKA)

[6252 rows x 1 columns]


In [115]:
# Función para eliminar filas cuyas variables no son paises
data_3 = lt.remove_non_countries(data_3)
print(data_3)

                 Country
0                    USA
1                    USA
2                    USA
3              AUSTRALIA
4                 MEXICO
...                  ...
6297           AUSTRALIA
6298           AUSTRALIA
6299                 USA
6300              PANAMA
6301  CEYLON (SRI LANKA)

[6179 rows x 1 columns]


In [116]:
# Función para añadir columna con el value_count de 'Country'
data_3 = lt.count_dataframe(data_3, column='Country', count_name='Attacks')
print(data_3)

                            Country  Attacks
0                               USA     2229
1                         AUSTRALIA     1338
2                      SOUTH AFRICA      579
3                  PAPUA NEW GUINEA      134
4                       NEW ZEALAND      128
..                              ...      ...
186  FEDERATED STATES OF MICRONESIA        1
187                            PERU        1
188                        PARAGUAY        1
189         PALESTINIAN TERRITORIES        1
190                     PHILIPPINES        1

[191 rows x 2 columns]


In [117]:
data_3['Country'].unique()

array(['USA', 'AUSTRALIA', 'SOUTH AFRICA', 'PAPUA NEW GUINEA',
       'NEW ZEALAND', 'BRAZIL', 'BAHAMAS', 'MEXICO', 'ITALY', 'FIJI',
       'PHILIPPINES', 'REUNION', 'NEW CALEDONIA', 'INDIA', 'CUBA',
       'MOZAMBIQUE', 'SPAIN', 'EGYPT', 'CROATIA', 'JAPAN', 'PANAMA',
       'SOLOMON ISLANDS', 'IRAN', 'JAMAICA', 'GREECE', 'FRENCH POLYNESIA',
       'HONG KONG', 'INDONESIA', 'ENGLAND', 'COSTA RICA', 'BERMUDA',
       'VIETNAM', 'TONGA', 'SRI LANKA', 'VANUATU', 'FRANCE',
       'MARSHALL ISLANDS', 'CANADA', 'IRAQ', 'TURKEY', 'UNITED KINGDOM',
       'VENEZUELA', 'SENEGAL', 'NEW GUINEA', 'KENYA', 'MAURITIUS',
       'COLUMBIA', 'ECUADOR', 'TAIWAN', 'SCOTLAND', 'SIERRA LEONE',
       'SAMOA', 'SOUTH KOREA', 'CARIBBEAN SEA', 'TANZANIA', 'THAILAND',
       'MADAGASCAR', 'CHILE', 'ISRAEL', 'YEMEN ', 'CHINA', 'SEYCHELLES',
       'DOMINICAN REPUBLIC', 'SINGAPORE', 'BARBADOS', 'NEW BRITAIN',
       'KIRIBATI', 'NICARAGUA', 'LIBYA', 'OKINAWA', 'SOMALIA', 'MALTA',
       'AZORES', 'MALAYSIA', 'SA

In [118]:
# Exportar dataset 3
# data_3.to_csv("drive/MyDrive/Colab Notebooks/csv_data/attacks_hypothesis3.csv", index = False, encoding="ISO-8859-1")